In [ ]:
######### IMPORTANT ############
#First, upload requirements.txt from here: https://huggingface.co/Qwen/Qwen-VL-Chat-Int4/blob/main/requirements.txt

!pip install -r /content/requirements.txt
!pip install flask



In [ ]:
!pip install optimum gekko

!git clone https://github.com/JustinLin610/AutoGPTQ.git
%cd AutoGPTQ
!pip install -v .

In [ ]:
!pip install auto-gptq tiktoken transformers_stream_generator accelerate einops

In [ ]:
### Step 2: Setting up Flask and Ngrok

#Import the necessary modules and initialize Flask and Ngrok.

from flask import Flask, request, jsonify
import base64
from PIL import Image
import io
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

# Initialize the Flask application
app = Flask(__name__)


In [ ]:
### Step 3: Load the Qwen LLM Model

#Load the Qwen-VL-Chat LLM model.
!pip install accelerate
torch.manual_seed(1234)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL-Chat-Int4", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat-Int4", device_map="cuda", trust_remote_code=True).eval()




In [ ]:
### Step 4: Define the API Endpoint

#Create an endpoint to handle POST requests, decode the image, and process it with the LLM.

@app.route('/completion', methods=['POST'])
def process_image():
    try:
        data = request.get_json()

        # Extract the prompt and image data from the request
        prompt = data['prompt']
        image_data = data['image_data'][0]['data']  # Assuming one image per request for simplicity

        # Decode the image
        image_bytes = base64.b64decode(image_data)
        image = Image.open(io.BytesIO(image_bytes))
        image.save("temp_image.jpg")  # Temporarily save the image to process

        # Process with LLM
        query = tokenizer.from_list_format([
            {'image': 'temp_image.jpg'},
            {'text': prompt},
        ])
        response, history = model.chat(tokenizer, query=query, history=None)

        # Remove the temp image
        os.remove("temp_image.jpg")

        return jsonify({"content": response})

    except Exception as e:
        return jsonify({"error": str(e)})




In [ ]:
!rm -f nohup.out
!wget -c https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared
!nohup ./cloudflared tunnel --url http://localhost:8007 &

# Give it a moment to establish the tunnel
!sleep 8

# Retrieve the URL from the output
!cat nohup.out | grep -o 'https://[a-zA-Z0-9-]*\.trycloudflare.com'

In [ ]:
### Step 5: Start the Server

#Add the command to run the Flask app.

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8007)
